# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve

from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix


from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import ConfusionMatrixDisplay

import lightgbm as lgb

import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

from aux_func import evaluate_model

time: 343 ms (started: 2021-12-15 19:04:42 +01:00)


In [3]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")['fatality']
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")['fatality']

time: 407 ms (started: 2021-12-12 23:02:00 +01:00)


# LightGBM

LightGBM es un clasificador que utiliza técnicas homólogas a las que utiliza XGBoost, pero se encuentra más optimizado que XGBoost permitiendo una mayor velocidad de entrenamiento y mayor eficiencia.
A diferencia de XGBoost, utiliza aprendizaje en paralelo.

![Highway](https://programmerclick.com/images/609/b84dc6b1590fb03af6971b4761aecc19.png)

In [4]:
clf = lgb.LGBMClassifier(n_jobs=-1, random_state=0)
clf.fit(xtrain, ytrain)

LGBMClassifier(random_state=0)

time: 15.3 s (started: 2021-12-12 23:02:00 +01:00)


In [5]:
with open('../models/LightGBM.pickle', 'wb') as f:
    pickle.dump(clf, f)

time: 16 ms (started: 2021-12-12 23:02:15 +01:00)


In [6]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
with open('../models/LightGBM.pickle', 'rb') as f:
    clf = pickle.load(f)

time: 15 ms (started: 2021-12-12 23:02:16 +01:00)


Generamos las predicciones sobre los datos de validación y evaluamos el modelo.

In [7]:
ypred = clf.predict(xtest)
ypred_proba = clf.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.8485746431939847
Accuracy of the model: 0.9845726939893004

Classification report: 
              precision    recall  f1-score   support

           0       0.98      1.00      0.99    797650
           1       0.45      0.01      0.02     12472

    accuracy                           0.98    810122
   macro avg       0.72      0.50      0.51    810122
weighted avg       0.98      0.98      0.98    810122


time: 3.45 s (started: 2021-12-12 23:02:16 +01:00)


## Ajuste del umbral de predicción

Procedemos a ajustar el umbral de la predicción para obtener un mayor recall en la variable minoritaria.

In [8]:
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.014631, G-Mean=0.764
ROC-AUC score of the model: 0.8485746431939847
Accuracy of the model: 0.7598337040593885

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.76      0.86    797650
           1       0.05      0.77      0.09     12472

    accuracy                           0.76    810122
   macro avg       0.52      0.76      0.48    810122
weighted avg       0.98      0.76      0.85    810122


time: 1.64 s (started: 2021-12-12 23:02:19 +01:00)


Podemos observar como el ajuste del threshold dota al modelo de un mayor recall para los casos de la clase minoritaria, lo cual nos interesa desde un punto de vista práctico a pesar de reducir la precisión y accuracy del modelo. 

El modelo LightGBM ha devuelto muy buenos resultados. A su vez, el tiempo de ejecución necesario para entrenar el modelo ha sido ínfimo en comparación con los anteriores.

## Comprobación de overfitting

Comprobamos si el modelo sufre de overfitting, realizando una predicción sobre la serie de entrenamiento.

In [9]:
ypred = clf.predict(xtrain)
ypred_proba = clf.predict_proba(xtrain)

# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytrain, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytrain,ypred_new_threshold,ypred_proba)

Best Threshold=0.015574, G-Mean=0.770
ROC-AUC score of the model: 0.8550318306384617
Accuracy of the model: 0.7749530317655859

Classification report: 
              precision    recall  f1-score   support

           0       1.00      0.78      0.87   3192113
           1       0.05      0.76      0.09     48375

    accuracy                           0.77   3240488
   macro avg       0.52      0.77      0.48   3240488
weighted avg       0.98      0.77      0.86   3240488


time: 15 s (started: 2021-12-12 23:02:21 +01:00)


El modelo LightGBM ha sido sorprendentemente bueno, teniendo en cuenta que cumple tres condiciones indispensables a la hora de generar un modelo:

- Buenos resultados
- Bajo tiempo de ejecución
- Ausencia de overfitting